In [103]:
# Ex 4. Discrete evenet simualtion

import numpy as np
import math
import scipy.stats as stats

from pydantic import BaseModel
from bisect import insort
from math import factorial


In [118]:
class DiscreteEventSimulator:
    def __init__(self, service_units, waiting_capacity, mean_service_time, mean_time_between_arrivals):
        self.steps = 0 # Discrete number of simulation steps
        self.time = 0 # Continuous time

        self.service_units = service_units
        self.waiting_capacity = waiting_capacity

        self.all_events = []
        self.queue = [] # Those being processed # Size should be service_units - 1 since we always keep one outside the queue (currently being processed)
        self.buffer = [] # Meant to hold those not being processed
        self.completed_events = []

        self.blocked = 0

        self.arrival_process_model = stats.expon(scale=1/mean_time_between_arrivals)

        self.service_time_model = stats.expon(scale=1/mean_service_time)

        self.next_event = None
        self.next_service_event = None

    def get_next_event(self, new_event):
        return min(new_event, min(self.queue))

    def get_next_service(self):
        # Should just return the first in the queue, I mean just use a queue for god's sake
        return min(self.queue, default=Event())

    def create_new_event(self, time=0):
        event_arrival_time = time + self.arrival_process_model.rvs()
        event_id = len(self.all_events)

        new_event = Event(id=event_id, arrival_time=event_arrival_time)

        return new_event
    
    def add_event_to_queue(self, event):
        event.required_service_time = self.service_time_model.rvs()
        event.completion_time = self.time + event.required_service_time
        insort(self.queue, event)

    def handle_new_event(self, event):
        if len(self.queue) < self.service_units:
            self.add_event_to_queue(event)
        elif len(self.buffer) < self.waiting_capacity:
            self.buffer.append(event)
        else:
            self.blocked += 1
    def simulate(self, duration=None, max_steps=None):
        
        if duration is not None:
            max_ = duration

        elif max_steps is not None:
            max_ = max_steps

        while self.time < max_:
            self.steps += 1
            self.new_event = self.create_new_event(self.time)

            if len(self.queue) == 0:
                self.time = self.new_event.arrival_time
                self.handle_new_event(self.new_event)
            
            else:
                while len(self.queue) != 0:
                    if self.new_event.arrival_time < self.queue[0].completion_time:
                        self.time = self.new_event.arrival_time
                        self.handle_new_event(self.new_event)
                        break
                    
                    else:
                        self.next_service_event = self.queue.pop(0)

                        time_delta = self.next_service_event.completion_time - self.time
                        self.time = self.next_service_event.completion_time

                        self.completed_events.append(self.next_service_event)

                        if self.buffer:
                            for i in self.buffer:
                                i.blocking_time += time_delta
                            
                            self.add_event_to_queue(self.buffer.pop(0))
                        
        print('Simulation terminated at time:', self.time)
                        
class Event(BaseModel):
    id: int = None
    required_service_time: float = np.inf
    arrival_time: float = np.inf
    completion_time: float = np.inf
    blocking_time: float = 0

    def __repr__(self): 
        return (self.id, self.required_service_time, self.arrival_time, self.completion_time)

    def __lt__(self, other):
        return self.completion_time < other.completion_time
    
    def __gt__(self, other):
        return self.completion_time > other.completion_time



In [127]:
max_service_units = 10
waiting_capacity = 0
mean_service_time = 1/8
mean_time_between_arrivals = 1/1


disc_event_sim = DiscreteEventSimulator(service_units=max_service_units, waiting_capacity=0, mean_service_time=mean_service_time, mean_time_between_arrivals=mean_time_between_arrivals)

disc_event_sim.simulate(duration=10000)

def erlang_b(traffic_intensity, num_servers):
    inv_b = sum((traffic_intensity ** k) / factorial(k) for k in range(num_servers + 1))
    return (traffic_intensity ** num_servers) / (factorial(num_servers) * inv_b)

def compare_with_erlang_b(arrival_rate, service_rate, service_units, blocked, completed_events):
    traffic_intensity = arrival_rate / service_rate
    erlang_b_blocking_prob = erlang_b(traffic_intensity, service_units)
    return erlang_b_blocking_prob

print("Simulated number of blocked:", disc_event_sim.blocked)

print("Simulated blocking probability:", disc_event_sim.blocked / (len(disc_event_sim.completed_events) + disc_event_sim.blocked))

print("Predicted blocking probability:", compare_with_erlang_b(mean_time_between_arrivals, mean_service_time, max_service_units, disc_event_sim.blocked, disc_event_sim.completed_events))     


Simulation terminated at time: 10000.288275906421
Simulated number of blocked: 1244
Simulated blocking probability: 0.1238797052380004
Predicted blocking probability: 0.1216610642529515


In [96]:
import numpy as np
from bisect import insort
from scipy import stats
from pydantic import BaseModel
from math import factorial

class Event(BaseModel):
    id: int = None
    required_service_time: float = np.inf
    arrival_time: float = np.inf
    completion_time: float = np.inf
    blocking_time: float = 0

    def __repr__(self): 
        return f"Event(id={self.id}, required_service_time={self.required_service_time}, arrival_time={self.arrival_time}, completion_time={self.completion_time}, blocking_time={self.blocking_time})"

    def __lt__(self, other):
        return self.completion_time < other.completion_time
    
    def __gt__(self, other):
        return self.completion_time > other.completion_time

class DiscreteEventSimulator:
    def __init__(self, service_units, waiting_capacity, arrival_rate, service_rate):
        self.steps = 0  # Discrete number of simulation steps
        self.time = 0  # Continuous time

        self.service_units = service_units
        self.waiting_capacity = waiting_capacity

        self.all_events = []
        self.queue = []  # Events being processed
        self.buffer = []  # Events waiting to be processed
        self.completed_events = []

        self.blocked = 0

        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.arrival_process_model = stats.expon(scale=1/arrival_rate)
        self.service_time_model = stats.expon(scale=1/service_rate)

    def create_new_event(self, time=0):
        event_arrival_time = time + self.arrival_process_model.rvs()
        event_id = len(self.all_events)

        new_event = Event(id=event_id, arrival_time=event_arrival_time)
        self.all_events.append(new_event)
        return new_event

    def add_event_to_queue(self, event):
        event.required_service_time = self.service_time_model.rvs()
        event.completion_time = self.time + event.required_service_time
        insort(self.queue, event)

    def handle_new_event(self, event):
        if len(self.queue) < self.service_units:
            self.add_event_to_queue(event)
        elif len(self.buffer) < self.waiting_capacity:
            self.buffer.append(event)
        else:
            self.blocked += 1

    def simulate(self, duration=None, max_steps=None):
        if duration is not None:
            max_ = duration
        elif max_steps is not None:
            max_ = max_steps
        else:
            raise ValueError("Either duration or max_steps must be provided")

        while (duration is not None and self.time < max_) or (max_steps is not None and self.steps < max_):
            self.steps += 1
            new_event = self.create_new_event(self.time)

            if len(self.queue) == 0:
                self.time = new_event.arrival_time
                self.handle_new_event(new_event)
            else:
                while len(self.queue) != 0:
                    if new_event.arrival_time < self.queue[0].completion_time:
                        self.time = new_event.arrival_time
                        self.handle_new_event(new_event)
                        break
                    else:
                        next_service_event = self.queue.pop(0)
                        time_delta = next_service_event.completion_time - self.time
                        self.time = next_service_event.completion_time
                        self.completed_events.append(next_service_event)

                        if self.buffer:
                            for buffered_event in self.buffer:
                                buffered_event.blocking_time += time_delta
                            self.add_event_to_queue(self.buffer.pop(0))

        print(f"Simulation finished at time {self.time} after {self.steps} steps")
        print(f"Completed events: {len(self.completed_events)}, Blocked events: {self.blocked}")
        
    def erlang_b(self, traffic_intensity, num_servers):
        inv_b = sum((traffic_intensity ** k) / factorial(k) for k in range(num_servers + 1))
        return (traffic_intensity ** num_servers) / (factorial(num_servers) * inv_b)

    def compare_with_erlang_b(self):
        traffic_intensity = self.arrival_rate / self.service_rate
        erlang_b_blocking_prob = self.erlang_b(traffic_intensity, self.service_units)
        simulation_blocking_prob = self.blocked / (self.blocked + len(self.completed_events))
        print(f"Erlang B Blocking Probability: {erlang_b_blocking_prob:.4f}")
        print(f"Simulation Blocking Probability: {simulation_blocking_prob:.4f}")

# Example usage
arrival_rate = 1  # Arrival rate (lambda)
service_rate = 1/8  # Service rate (mu)
service_units = 10  # Number of service units
waiting_capacity = 0  # Waiting buffer capacity

sim = DiscreteEventSimulator(service_units, waiting_capacity, arrival_rate, service_rate)
sim.simulate(duration=10000)
sim.compare_with_erlang_b()


Simulation finished at time 10001.390228684017 after 9945 steps
Completed events: 8744, Blocked events: 1190
Erlang B Blocking Probability: 0.1217
Simulation Blocking Probability: 0.1198


In [80]:
    def simulate(self, duration=None, max_steps=None):
        
        if duration is not None:
            max_ = duration

        elif max_steps is not None:
            max_ = max_steps

        while self.time < max_:
            self.steps += 1

            self.next_event = self.create_new_event(self.time)

            # Does this new event arrive before the first event can finish?
            if self.next_event.arrival_time < self.queue[0].completion_time:
                self.handle_new_event(self.next_event)

            # If not, the first event will finish
            else:
                self.next_service_event = self.queue.pop(0)

                time_delta = self.next_service_event.completion_time - self.time
                self.time = self.next_service_event.completion_time

                self.completed_events.append(self.next_service_event)

                if self.buffer:
                    for i in self.buffer:
                        i.blocking_time += time_delta
                    
                
                self.queue.append(self.next_event)

            # TODO: Required that we have self.next_event? Why keep as self variable?
            self.next_event = self.create_new_event(self.time)
            self.next_service_event = self.queue.pop(0)

            while self.next_service_event.completion_time < self.next_event.arrival_time and self.queue:

                time_delta = self.next_service_event.completion_time - self.time
                self.time = self.next_service_event.completion_time

                self.completed_events.append(self.next_service_event)

                if self.buffer:
                    for i in self.buffer:
                        i.blocking_time += time_delta
                    
                    self.add_event_to_queue(self.buffer.pop(0))
                
                self.next_service_event = self.queue.pop(0) 

            if len(self.queue) < (self.service_units - 1):
                self.add_event_to_queue(self.next_event)
            
            elif len(self.buffer < self.waiting_capacity):
                self.buffer.append(self.next_event)
            
            else:
                self.blocked += 1


In [ ]:
len([])

0